In [0]:
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
from torch import nn, optim
import torchvision
import torch.utils.data
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import helper
import time
import copy
import seaborn as sns
import numpy as np
from PIL import Image
from collections import OrderedDict

Preprocessing data

In [2]:
!git clone https://github.com/tobywynne-mellor/recycle-bot

Cloning into 'recycle-bot'...
remote: Enumerating objects: 2595, done.
remote: Counting objects: 100% (2595/2595), done.
remote: Compressing objects: 100% (2574/2574), done.
remote: Total 2595 (delta 33), reused 2571 (delta 18), pack-reused 0
Receiving objects: 100% (2595/2595), 40.45 MiB | 21.60 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [0]:
data_transforms = {
    'validset': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
}

In [0]:
dataset = datasets.ImageFolder('/content/recycle-bot/Waste ',transform = data_transforms['validset'])


Dividing the dataset into test, validation, train sets


In [0]:
trainset, validset = random_split(dataset, (int(len(dataset)*0.8), int(len(dataset)*0.2)))

In [0]:
dataloader_train = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle = True)
dataloader_valid = torch.utils.data.DataLoader(validset, batch_size=32, shuffle = False)

In [0]:
dataloader = {
    'train' : dataloader_train,
    'valid' : dataloader_valid
}

Input a pretrained transfer learning model

In [8]:
model = models.vgg19(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:15<00:00, 36.4MB/s]


In [0]:
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, 4096)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(4096, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))

In [0]:
for param in model.parameters():
    param.requires_grad = False

In [0]:
model.classifier = classifier

In [12]:
model.classifier

Sequential(
  (fc1): Linear(in_features=25088, out_features=4096, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=4096, out_features=102, bias=True)
  (output): LogSoftmax()
)

Train the model

In [0]:
def train_model(model, dataloaders, criteria, optimizer, scheduler,    
                                      num_epochs=25, device='cuda'):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criteria(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [0]:
criteria = nn.NLLLoss()
optim = torch.optim.Adam(model.classifier.parameters(), lr=0.001)
sched = torch.optim.lr_scheduler.StepLR(optim, step_size=4, gamma=0.1)
eps=10

In [15]:
model.cuda()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

Display model accuracy

In [16]:
train_model(model,dataloader,criteria,optim,sched,eps)

Epoch 0/9
----------


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Loss: 2.2552 Acc: 0.6930
valid Loss: 0.5237 Acc: 0.8117

Epoch 1/9
----------
train Loss: 0.1327 Acc: 0.9498
valid Loss: 0.6690 Acc: 0.8201

Epoch 2/9
----------
train Loss: 0.0278 Acc: 0.9937
valid Loss: 0.6795 Acc: 0.8431

Epoch 3/9
----------
train Loss: 0.0063 Acc: 0.9995
valid Loss: 0.6358 Acc: 0.8494

Epoch 4/9
----------
train Loss: 0.0035 Acc: 0.9995
valid Loss: 0.6353 Acc: 0.8556

Epoch 5/9
----------
train Loss: 0.0031 Acc: 0.9995
valid Loss: 0.6243 Acc: 0.8536

Epoch 6/9
----------
train Loss: 0.0030 Acc: 0.9995
valid Loss: 0.6369 Acc: 0.8556

Epoch 7/9
----------
train Loss: 0.0026 Acc: 0.9995
valid Loss: 0.6360 Acc: 0.8556

Epoch 8/9
----------
train Loss: 0.0025 Acc: 0.9995
valid Loss: 0.6335 Acc: 0.8556

Epoch 9/9
----------
train Loss: 0.0024 Acc: 0.9995
valid Loss: 0.6323 Acc: 0.8536

Training complete in 7m 2s
Best val Acc: 0.855649


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

Input validation set and check models performance

In [0]:
def calc_accuracy(model, dataloaders, data, cuda=False):
    model.eval()
    model.to(device='cuda')    
    material = []
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(dataloaders[data]):
            if cuda:
                inputs, labels = inputs.cuda(), labels.cuda()
            # obtain the outputs from the model
            outputs = model.forward(inputs)
            # max provides the (maximum probability, max value)
            _, predicted = outputs.max(dim=1)
            material.append(predicted)
            # check the 
            if idx == 0:
                print(predicted) #the predicted class
                print(torch.exp(_)) # the predicted probability
            equals = predicted == labels.data
            if idx == 0:
                print(equals)
            print(equals.float().mean())
    return material

In [18]:
first_batch = calc_accuracy(model, dataloader, 'valid', True)[0]

tensor([3, 1, 2, 4, 2, 2, 2, 3, 0, 3, 0, 2, 4, 3, 0, 0, 4, 4, 0, 3, 3, 3, 4, 3,
        2, 4, 3, 1, 3, 1, 3, 0], device='cuda:0')
tensor([1.0000, 0.9205, 0.9996, 0.9999, 1.0000, 0.9882, 1.0000, 1.0000, 1.0000,
        1.0000, 0.9998, 0.9951, 0.9680, 0.9804, 0.9476, 1.0000, 0.9970, 0.7002,
        1.0000, 0.9820, 0.9998, 1.0000, 0.3549, 1.0000, 1.0000, 0.9878, 0.6741,
        0.9824, 0.9999, 0.9607, 0.9631, 1.0000], device='cuda:0')
tensor([ True,  True,  True,  True,  True,  True, False,  True,  True,  True,
         True,  True, False,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True], device='cuda:0')
tensor(0.9375, device='cuda:0')
tensor(0.8438, device='cuda:0')
tensor(0.8750, device='cuda:0')
tensor(0.8125, device='cuda:0')
tensor(0.8750, device='cuda:0')
tensor(0.8750, device='cuda:0')
tensor(0.8125, device='cuda:0')
tensor(0.8125, device='cuda:0')
tensor(0.8125, device='cuda:0')
te

Convert material representation from numerical to text (For a batch)




In [0]:
def BatchNumToMaterial(inputted_data):
  material2 = []
  for i in range (0,len(first_batch)):
    if inputted_data[i].item() == 0:
      material2.append('cardboard')
    elif inputted_data[i].item() == 1:
      material2.append('glass')
    elif inputted_data[i].item() == 2:
      material2.append('metal')
    elif inputted_data[i].item() == 3:
      material2.append('paper')
    elif inputted_data[i].item() == 4:
      material2.append('plastic')
  return material2


In [20]:
BatchNumToMaterial(first_batch)

['paper',
 'glass',
 'metal',
 'plastic',
 'metal',
 'metal',
 'metal',
 'paper',
 'cardboard',
 'paper',
 'cardboard',
 'metal',
 'plastic',
 'paper',
 'cardboard',
 'cardboard',
 'plastic',
 'plastic',
 'cardboard',
 'paper',
 'paper',
 'paper',
 'plastic',
 'paper',
 'metal',
 'plastic',
 'paper',
 'glass',
 'paper',
 'glass',
 'paper',
 'cardboard']

Convert material representation from numerical to text (For a single value)

In [0]:
def SingleNumToMaterial(value):
    if torch.tensor(value).item() == 0:
      return('cardboard')
    elif torch.tensor(value).item() == 1:
      return('glass')
    elif torch.tensor(value).item() == 2:
      return('metal')
    elif torch.tensor(value).item() == 3:
      return('paper')
    elif torch.tensor(value).item() == 4:
      return('plastic')

Input a path to image location (Ground truth isnt actually necessary so it has a default value where if not specified then the True/False output should be ignored) and get an text material output. 

In [0]:
def image_predictor(path,ground_truth=0):
  image = Image.open(path)
  imagetensor = data_transforms['validset'](image)
  dataset_single = torch.utils.data.TensorDataset(imagetensor[None], torch.tensor([ground_truth]))
  dataloader_valid_single = torch.utils.data.DataLoader(dataset_single, batch_size=1, shuffle = False)
  dataloader = {
    'train' : dataloader_valid_single,
    'valid' : dataloader_valid_single
  }
  value2 = calc_accuracy(model,dataloader,'valid',True)[0].item()
  return SingleNumToMaterial(value2)


Image for each material

In [24]:
image_predictor('/content/plasticbottleempty.jpg')

tensor([4], device='cuda:0')
tensor([0.6519], device='cuda:0')
tensor([False], device='cuda:0')
tensor(0., device='cuda:0')


'plastic'

In [25]:
image_predictor('/content/coffecup.jpg')

tensor([0], device='cuda:0')
tensor([0.4226], device='cuda:0')
tensor([True], device='cuda:0')
tensor(1., device='cuda:0')


'cardboard'

In [26]:
image_predictor('/content/metalcan.jpg')

tensor([2], device='cuda:0')
tensor([0.9966], device='cuda:0')
tensor([False], device='cuda:0')
tensor(0., device='cuda:0')


'metal'

In [27]:
image_predictor('/content/papertest.jpg')

tensor([3], device='cuda:0')
tensor([0.9819], device='cuda:0')
tensor([False], device='cuda:0')
tensor(0., device='cuda:0')


'paper'

In [32]:
image_predictor('/content/glass.jpg')

tensor([1], device='cuda:0')
tensor([1.], device='cuda:0')
tensor([False], device='cuda:0')
tensor(0., device='cuda:0')


'glass'

Save model and use flask to link Frontend and Backend.

In [0]:
torch.save(model.state_dict(), '/themodel.pth')